In [52]:
##############
#Dependencies#
##############
%matplotlib notebook
from selenium import webdriver
from datetime import datetime
import pandas
import os
import numpy as np
import time
import folium


In [162]:
def import_summary():
    global route_df
    route_df= pandas.read_csv('Data/route_summary.csv')
    route_df = route_df.set_index("Station")

    route_df= route_df.fillna(0)
    route_df = route_df.drop("Unnamed: 0", axis=1)
    route_df

In [186]:
route_df


,Heading,Time,Order,Speed,Flow,Lat,Lng
Station,,,,,,,
S10,SB,Wed Oct 10 16:41:59 CDT 2018,54,82,930,44.92003,-93.27488
S10,SB,Wed Oct 10 16:42:29 CDT 2018,54,70,840,44.92003,-93.27488
S10,SB,Wed Oct 10 16:42:59 CDT 2018,54,76,1080,44.92003,-93.27488
S10,SB,Wed Oct 10 16:43:29 CDT 2018,54,71,840,44.92003,-93.27488
S10,SB,Wed Oct 10 16:43:59 CDT 2018,54,61,690,44.92003,-93.27488
S10,SB,Wed Oct 10 16:44:59 CDT 2018,54,70,390,44.92003,-93.27488
S10,SB,Wed Oct 10 16:45:29 CDT 2018,54,68,720,44.92003,-93.27488
S10,SB,Wed Oct 10 16:45:59 CDT 2018,54,61,780,44.92003,-93.27488
S10,SB,Wed Oct 10 16:46:29 CDT 2018,54,70,810,44.92003,-93.27488


In [273]:
def Timed_Map(Datetimestring):
    route_timed = route_df.reset_index().set_index(["Time"])
    route_timed = route_timed.loc[[Datetimestring]]
    route_timed = route_timed.reset_index().set_index(["Station"])
    grab_timed_data(route_timed)
    

In [286]:
def grab_timed_data(DataFrame):
    global Results_List
    route = DataFrame.groupby('Station').head(1).index.values
    Results = {}

    for station in route:
        try:
            Flow =  float(DataFrame.loc[station,'Flow'])
            Speed = int(DataFrame.loc[station,'Speed'])
            Lng =DataFrame.loc[station,'Lng']
            Lat = DataFrame.loc[station,'Lat']
            Order = DataFrame.loc[station,'Order'].astype(dtype="int")
            Heading = DataFrame.loc[station,'Heading']
        



            Results.update({station : {'Station' :station,
                "Heading" : Heading,
                "Order" : Order,
                "Current Speed" : Speed,
                "Current Flow" : Flow,
        
                "Lat":Lat, 
                "Lng":Lng}})


        except ValueError as v:
            print(f"{station} {v}")
    Results = pandas.DataFrame(Results).T
    Results = Results.sort_values(['Heading', 'Order'])
    Results = Results.set_index(['Heading', 'Order'], drop=True)
    Results.head()
    ResultsNB = Results.xs('NB', level='Heading')
    ResultsSB = Results.xs('NB', level='Heading')
    Results_List= {"NB":ResultsNB,"SB":ResultsSB}

In [300]:
Results_List


{'NB':       Current Flow Current Speed      Lat      Lng Station
 Order                                                     
 1             1200            19  45.0724 -93.1852    S664
 6             1280            33   45.066  -93.186    S659
 8             1000            60  45.0725 -93.1856    S685
 10            1280            23  45.0485 -93.1887    S657
 12            1360            25  45.0417 -93.1893    S656
 15            1240            31  45.0279 -93.1908    S654
 17             880            60  45.0222 -93.1897    S653
 19            1440            60  45.0158 -93.1875    S652
 22            1320            59  45.0117 -93.1879    S575
 23            1440            23  45.0099 -93.2003    S574
 24            1640            60  45.0101 -93.2004    S577
 25            1200            11  45.0089 -93.2023    S587
 26            1320            27  45.0052 -93.2095    S573
 29            1520            71  45.0007 -93.2235    S572
 31            1680            50 

In [272]:
timenow = datetime.now().strftime("%b%d_%H%M%S")

In [297]:
def mapping():
            
    global folium_map
    for result in Results_list:

        x = int(len(Results_List[result]['Station']) / 2)

        folium_map = folium.Map((Results_List[result].iloc[x, 2],ResultsNB.iloc[x,3]),

                                zoom_start=11,
                                 tiles="CartoDB positron")
        Features = []
        Last_Sensor = []
        for index, row in Results_List[result].iterrows():
            if row['Current Speed'] < 15:
                color = "#ff0000"
            elif row['Current Speed'] >= 15 and row['Current Speed'] < 30:
                color =  "#ffa500"
            elif row['Current Speed'] >= 30 and row['Current Speed'] < 55:
                color = "#ffff00"
            else:
                color = "#008000"

            weight = row['Current Flow'] / 200
            if row['Current Flow'] == 0:
                weight = 1
                color = "#808080"




            Current_Sensor = (row['Lat'], row['Lng'])

            if Last_Sensor == [] :
                Last_Sensor = (row['Lat'], row['Lng'])
            else:
                if row['Current Flow'] != 0:
                    weight = row['Current Flow'] / 200

                folium.PolyLine([Current_Sensor,Last_Sensor],
                 weight=weight,color=color,
                                popup=f"Time:{timenow} Speed:{row['Current Speed']} Flow: {row['Current Flow']}").add_to(folium_map)

                Last_Sensor = (row['Lat'], row['Lng'])

            folium.CircleMarker(location=(Current_Sensor),
                                radius=3,
                                popup=("station ="  + row['Station']), fill=False).add_to(folium_map)

        folium_map.save(f"results/maps/routemap_temp.html")
        print(f'Map saved at results/maps/routemap_temp.html')
        delay=7
        fn=f'results/maps/routemap_temp.html'
        tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

        browser = webdriver.Firefox()
        browser.get(tmpurl)
        #Give the map tiles some time to load
        time.sleep(delay)
        browser.save_screenshot(f'results/maps/{result}routemap{timenow}.png')
        browser.quit()
        print(f' Map Converted -->> results/maps/{result}routemap{timenow}')
    

In [301]:
import_summary()
Timed_Map("Wed Oct 10 16:41:59 CDT 2018")
mapping()


NameError: name 'Results_list' is not defined

In [302]:
Results_list

NameError: name 'Results_list' is not defined